# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of the project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing the Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance on your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity')
except Exception as e :
    print(e)

### Create tables to run the following queries. With Apache Cassandra, database tables are modeled based on the queries you want to run.

## Create queries to ask the following three questions of the data
### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 
### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Each query requires three basic steps: create a table, shape an incoming row of data, insert that row into the table

### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [9]:
# creat table to answer the query above
query = "CREATE TABLE IF NOT EXISTS music_app_history"

# partitio key is for filtering, clustering key is for sorting. 
# here, two filtering columns, so two combined columns as one partition key 
query = query + "(sessionId int, itemInSession int, artist text, song text, length decimal, \
PRIMARY KEY((sessionId, itemInSession)) )"     
# has to use decimal as the data type for length
# comma for each column. 
# Do not include data type into your pk

try:
    session.execute(query)
except Exception as e:
    print(e)                 

In [10]:
# set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)  # csv.reader() returns reader object which iterates over lines in the given CSV file
    next(csvreader) # skip header , goes to the next item from the iterator
    for line in csvreader:
        #print(line)
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history(sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        # when you grab data as CSV,  everything by default is a string
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [17]:
# Test query Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, 
# itemInSession = 4
query1 = "SELECT * FROM music_app_history WHERE sessionId = 338 AND itemInSession = 4"
try: 
    rows = session.execute(query1)
except Exception as e:
    print(e)

data1 = []
for row in rows:
    #print(row.artist,' ', row.song, ' ', row.length)
    # treat the values as a tuple, otherwise it gives " ValueError: Shape of passed values is (1, 3), indices imply (3, 3)"
    data1.append((row.artist, row.song, row.length))
    
df1 = pd.DataFrame(data1, columns = ['Artist_Name', 'Song_Title', 'Song_Length'])
df1
    

,Artist_Name,Song_Title,Song_Length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
# create table for the query
query = "CREATE TABLE IF NOT EXISTS music_app_history2"

# partitio key is for filtering, clustering key is for sorting. 
# two filtering conditions as one partition key, only one clustering key needed based on the query, 
# do not need to use "order by" in the query later coz clustering column will automatically sort
query = query + "(userId int, sessionId int, itemInSession int, song text, artist text, first_name text, last_name text,\
PRIMARY KEY((userId, sessionId), itemInSession) )"   

# has to use decimal as the data type for length
# comma for each column. 
# Do not include data type into your pk

try:
    session.execute(query)
except Exception as e:
    print(e)                      

In [13]:
# set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)  # csv.reader() returns reader object which iterates over lines in the given CSV file
    next(csvreader) # skip header , goes to the next item from the iterator
    for line in csvreader:
        #print(line)
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history2 (userId, sessionId, itemInSession, song, artist, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        # when you grab data as CSV,  everything by default is a string
        session.execute(query, ( int(line[10]), int(line[8]), int(line[3]), line[9], line[0], line[1], line[4]) )

In [18]:
# run query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) 
# for userid = 10, sessionid = 182
query = "SELECT * FROM music_app_history2 WHERE userId= 10 AND sessionId= 182"
try: 
    rows = session.execute(query)
except Exception as e:
    print(e)

data2 = []
for row in rows:
    #print(row.artist,' ', row.song, ' ', row.first_name,' ', row.last_name )
    # treat the values as a tuple, otherwise it gives " ValueError: Shape of passed values is (1, 4), indices imply (4, 4)"
    data2.append((row.artist, row.song, row.first_name, row.last_name))
df2 = pd.DataFrame(data2, columns = ['Artist_Name', 'Song_Title', 'User_Fname', 'User_Lname'])
df2

,Artist_Name,Song_Title,User_Fname,User_Lname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [20]:
# create table for the query
query = "CREATE TABLE IF NOT EXISTS music_app_history3"

# you can build the table according to the query. You may add sth not in the query if needed. Here userId
# partitio key is for filtering, clustering key is for sorting. 
# song title is for filtering , userId is for sorting 
query = query + "( song text, userId int, first_name text, last_name text, PRIMARY KEY( song, userId) )"     
# has to use decimal as the data type for length
# comma for each column. 
# Do not include data type into your pk

try:
    session.execute(query)
except Exception as e:
    print(e)                        

In [21]:
# set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)  # csv.reader() returns reader object which iterates over lines in the given CSV file
    next(csvreader) # skip header , goes to the next item from the iterator
    for line in csvreader:
        #print(line)
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history3(song, userId, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        # when you grab data as CSV,  everything by default is a string
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [24]:
# run query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# The partition key determines on which node each partition will be stored. So if you don't specify the partition key, 
# then Cassandra has to filter all the partitions of this table on all the nodes to find matching data.
# That's why you have to say ALLOW FILTERING, since that operation is very inefficient and is discouraged.

query = "SELECT * FROM music_app_history3 WHERE song='All Hands Against His Own' "
try: 
    rows = session.execute(query)
except Exception as e:
    print(e)
    
data3 = []    
for row in rows:
    #print(row.first_name,' ', row.last_name )
    # treat the values as a tuple, otherwise it gives " ValueError: Shape of passed values is (1, 2), indices imply (2, 2)"
    data3.append((row.first_name, row.last_name))
df3 = pd.DataFrame(data3, columns = [ 'User_Fname', 'User_Lname'])
df3

,User_Fname,User_Lname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [27]:
def drop_table(query):
    try:
        session.execute(query)
    except Exception as e:
        print(e)

In [28]:
query1 = "DROP TABLE music_app_history "
query2 = "DROP TABLE music_app_history2 "
query3 = "DROP TABLE music_app_history3 "

query_list = [query1, query2, query3]
for query in query_list:
    drop_table(query)

### Close the session and cluster connection¶

In [29]:
session.shutdown()
cluster.shutdown()

### Reference:
* https://blog.knoldus.com/cassandra-data-modeling-primary-clustering-partition-compound-keys/
* http://zetcode.com/python/csv/